# LSTM을 이용한 텍스트 생성
- 데이터 다운로드: 캐글 New York Times Comments

1. 데이터 전처리

In [2]:
import numpy as np
import pandas as pd
from string import punctuation

from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

Using TensorFlow backend.


In [22]:
df=pd.read_csv('../dataset/ArticlesApril2018.csv', encoding = "latin")
df.head()

,articleID,articleWordCount,byline,documentType,headline,keywords,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL
0,5adf6684068401528a2aa69b,781,By JOHN BRANCH,article,Former N.F.L. Cheerleadersâ Settlement Offer...,"['Workplace Hazards and Violations', 'Football...",68,Sports,0,2018-04-24 17:16:49,Pro Football,"âI understand that they could meet with us, ...",The New York Times,News,https://www.nytimes.com/2018/04/24/sports/foot...
1,5adf653f068401528a2aa697,656,By LISA FRIEDMAN,article,E.P.A. to Unveil a New Rule. Its Effect: Less ...,"['Environmental Protection Agency', 'Pruitt, S...",68,Climate,0,2018-04-24 17:11:21,Unknown,The agency plans to publish a new regulation T...,The New York Times,News,https://www.nytimes.com/2018/04/24/climate/epa...
2,5adf4626068401528a2aa628,2427,By PETE WELLS,article,"The New Noma, Explained","['Restaurants', 'Noma (Copenhagen, Restaurant)...",66,Dining,0,2018-04-24 14:58:44,Unknown,Whatâs it like to eat at the second incarnat...,The New York Times,News,https://www.nytimes.com/2018/04/24/dining/noma...
3,5adf40d2068401528a2aa619,626,By JULIE HIRSCHFELD DAVIS and PETER BAKER,article,Unknown,"['Macron, Emmanuel (1977- )', 'Trump, Donald J...",68,Washington,0,2018-04-24 14:35:57,Europe,President Trump welcomed President Emmanuel Ma...,The New York Times,News,https://www.nytimes.com/2018/04/24/world/europ...
4,5adf3d64068401528a2aa60f,815,By IAN AUSTEN and DAN BILEFSKY,article,Unknown,"['Toronto, Ontario, Attack (April, 2018)', 'Mu...",68,Foreign,0,2018-04-24 14:21:21,Canada,"Alek Minassian, 25, a resident of Torontoâs ...",The New York Times,News,https://www.nytimes.com/2018/04/24/world/canad...


In [4]:
print('열의 개수: ', len(df.columns))
print(df.columns)

열의 개수:  15
Index(['articleID', 'articleWordCount', 'byline', 'documentType', 'headline',
       'keywords', 'multimedia', 'newDesk', 'printPage', 'pubDate',
       'sectionName', 'snippet', 'source', 'typeOfMaterial', 'webURL'],
      dtype='object')


In [5]:
# headline 열의 데이터만 사용, Null 검사
df['headline'].isnull().values.any()

False

In [6]:
# headline 열의 데이터로 리스트 생성
headline = [title for title in df.headline.values]
headline[:5]

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'Unknown',
 'Unknown']

In [7]:
len(headline)

1324

In [8]:
# 노이즈 데이터('Unknown') 제거
headline = [title for title in headline if title != 'Unknown']
len(headline)

1214

In [9]:
# 구두점 제거와 소문자화를 위한 함수
def repreprocessing(s):
    s=s.encode("utf8").decode("ascii",'ignore')
    return ''.join(c for c in s if c not in punctuation).lower()

In [10]:
text = [repreprocessing(x) for x in headline]
text[:5]

['former nfl cheerleaders settlement offer 1 and a meeting with goodell',
 'epa to unveil a new rule its effect less science in policymaking',
 'the new noma explained',
 'how a bag of texas dirt  became a times tradition',
 'is school a place for selfexpression']

In [11]:
# 단어 집합(vocabulary)을 만들고 크기를 확인
t = Tokenizer()
t.fit_on_texts(text)
vocab_size = len(t.word_index) + 1
print('단어 집합의 크기 : %d' % vocab_size)

단어 집합의 크기 : 3494


In [12]:
sequences = []
for line in text: # 1,214 개의 샘플에 대해서 샘플을 1개씩 가져온다.
    encoded = t.texts_to_sequences([line])[0] # 각 샘플에 대한 정수 인코딩
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)

sequences[:11] # 11개의 샘플 출력

[[99, 269],
 [99, 269, 371],
 [99, 269, 371, 1115],
 [99, 269, 371, 1115, 582],
 [99, 269, 371, 1115, 582, 52],
 [99, 269, 371, 1115, 582, 52, 7],
 [99, 269, 371, 1115, 582, 52, 7, 2],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10, 1116],
 [100, 3]]

In [13]:

index_to_word = {}
for key, value in t.word_index.items(): # 인덱스를 단어로 바꾸기 위해 index_to_word를 생성
    index_to_word[value] = key

print('빈도수 상위 1번 단어 :', index_to_word[1])
print('빈도수 상위 582번 단어 :', index_to_word[582])

빈도수 상위 1번 단어 : the
빈도수 상위 582번 단어 : offer


In [14]:
max_len=max(len(s) for s in sequences)
print('샘플의 최대 길이 :', max_len)

샘플의 최대 길이 : 24


In [15]:
# 전체 샘플의 길이를 24(가장 긴 샘플의 길이)로 패딩
# 'pre' 옵션을 주면 앞을 0으로 패딩
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')
print(sequences[:3])

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0   99  269]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0   99  269  371]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0   99  269  371 1115]]


In [16]:
X = sequences[:,:-1]
y = sequences[:,-1]

In [17]:
# 레이블 데이터 y에 대해서 원-핫 인코딩을 수행
y = to_categorical(y, num_classes=vocab_size)

In [18]:
X.shape, y.shape

((7803, 23), (7803, 3494))

# 2. 모델 설계 및 학습

In [19]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Embedding, Dense, LSTM

In [20]:

# 임베딩 벡터는 10차원, 은닉 상태 크기는 128
embedding = Embedding(vocab_size, 10, input_length=max_len-1, name="Embedding_Layer")
lstm = LSTM(128, name="LSTM_Layer")
output = Dense(vocab_size, activation='softmax', name="Output_Layer")

In [21]:

model = Sequential()
model.add(embedding)
model.add(lstm)
model.add(output)
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Embedding_Layer (Embedding)  (None, 23, 10)            34940     
_________________________________________________________________
LSTM_Layer (LSTM)            (None, 128)               71168     
_________________________________________________________________
Output_Layer (Dense)         (None, 3494)              450726    
Total params: 556,834
Trainable params: 556,834
Non-trainable params: 0
_________________________________________________________________


In [23]:
model.compile(loss='categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

In [24]:
history = model.fit(X, y, epochs=200, verbose=2)

C:\ProgramData\Anaconda3\envs\tf2\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/200
 - 4s - loss: 7.6350 - accuracy: 0.0278
Epoch 2/200
 - 4s - loss: 7.1061 - accuracy: 0.0313
Epoch 3/200
 - 4s - loss: 6.9698 - accuracy: 0.0338
Epoch 4/200
 - 4s - loss: 6.8504 - accuracy: 0.0413
Epoch 5/200
 - 4s - loss: 6.7056 - accuracy: 0.0461
Epoch 6/200
 - 4s - loss: 6.5479 - accuracy: 0.0481
Epoch 7/200
 - 4s - loss: 6.3622 - accuracy: 0.0525
Epoch 8/200
 - 4s - loss: 6.1615 - accuracy: 0.0559
Epoch 9/200
 - 4s - loss: 5.9610 - accuracy: 0.0627
Epoch 10/200
 - 4s - loss: 5.7711 - accuracy: 0.0668
Epoch 11/200
 - 4s - loss: 5.5885 - accuracy: 0.0681
Epoch 12/200
 - 4s - loss: 5.4173 - accuracy: 0.0782
Epoch 13/200
 - 4s - loss: 5.2563 - accuracy: 0.0830
Epoch 14/200
 - 4s - loss: 5.1070 - accuracy: 0.0878
Epoch 15/200
 - 4s - loss: 4.9641 - accuracy: 0.1009
Epoch 16/200
 - 4s - loss: 4.8296 - accuracy: 0.1062
Epoch 17/200
 - 4s - loss: 4.6950 - accuracy: 0.1162
Epoch 18/200
 - 4s - loss: 4.5704 - accuracy: 0.1306
Epoch 19/200
 - 4s - loss: 4.4463 - accuracy: 0.1457
Ep

Epoch 155/200
 - 4s - loss: 0.2995 - accuracy: 0.9163
Epoch 156/200
 - 4s - loss: 0.3064 - accuracy: 0.9161
Epoch 157/200
 - 4s - loss: 0.3029 - accuracy: 0.9168
Epoch 158/200
 - 4s - loss: 0.2976 - accuracy: 0.9159
Epoch 159/200
 - 4s - loss: 0.2962 - accuracy: 0.9153
Epoch 160/200
 - 4s - loss: 0.2920 - accuracy: 0.9171
Epoch 161/200
 - 4s - loss: 0.2899 - accuracy: 0.9153
Epoch 162/200
 - 4s - loss: 0.2886 - accuracy: 0.9152
Epoch 163/200
 - 4s - loss: 0.2888 - accuracy: 0.9159
Epoch 164/200
 - 4s - loss: 0.2866 - accuracy: 0.9157
Epoch 165/200
 - 4s - loss: 0.2843 - accuracy: 0.9172
Epoch 166/200
 - 4s - loss: 0.2829 - accuracy: 0.9164
Epoch 167/200
 - 4s - loss: 0.2940 - accuracy: 0.9146
Epoch 168/200
 - 4s - loss: 0.2837 - accuracy: 0.9167
Epoch 169/200
 - 4s - loss: 0.2798 - accuracy: 0.9166
Epoch 170/200
 - 4s - loss: 0.2784 - accuracy: 0.9163
Epoch 171/200
 - 4s - loss: 0.2781 - accuracy: 0.9180
Epoch 172/200
 - 4s - loss: 0.2775 - accuracy: 0.9161
Epoch 173/200
 - 4s - loss: 

In [25]:
def sentence_generation(model, t, current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word # 처음 들어온 단어도 마지막에 같이 출력하기위해 저장
    sentence = ''
    for _ in range(n): # n번 반복
        encoded = t.texts_to_sequences([current_word])[0] # 현재 단어에 대한 정수 인코딩
        encoded = pad_sequences([encoded], maxlen=23, padding='pre') # 데이터에 대한 패딩
        result = model.predict_classes(encoded, verbose=0)
          # 입력한 X(현재 단어)에 대해서 y를 예측하고 y(예측한 단어)를 result에 저장.
        for word, index in t.word_index.items(): 
            if index == result: # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
                break # 해당 단어가 예측 단어이므로 break
        current_word = current_word + ' '  + word # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        sentence = sentence + ' ' + word # 예측 단어를 문장에 저장

    sentence = init_word + sentence
    return sentence

In [26]:
print(sentence_generation(model, t, 'i', 10))
# 임의의 단어 'i'에 대해서 10개의 단어를 추가 생성
print(sentence_generation(model, t, 'how', 10))
# 임의의 단어 'how'에 대해서 10개의 단어를 추가 생성
print(sentence_generation(model, t, 'former', 10))

i cant jump ship from facebook yet control of wwi up
how to make a crossword puzzle virtual the initials battle over
former cheney aide is pardoned by trump in cia leak case
